In [2]:
import math
from pomegranate import *

In [24]:
import csv

with open('INTC.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    aberturas = []
    fechamentos = []
    
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are: {", ".join(row)}')
            line_count += 1
        else:
            #print(row)
            aberturas.append(row[1])
            fechamentos.append(row[4])
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are: Date, Open, High, Low, Close, Adj Close, Volume
Processed 252 lines.


In [20]:
#print(aberturas)

In [21]:
#print(fechamentos)

In [33]:
# calculo da probabilidade de subir
count = 0

for i in range(0, line_count-1):
    if aberturas[i] < fechamentos[i]:
        count += 1
        
prob_subir = count/(line_count-1)

print(prob_subir)

0.5378486055776892


In [26]:
estado_hoje = DiscreteDistribution( { 'prob_subir': prob_subir, 'prob_descer': 1-prob_subir} )

In [30]:
#print(estado_hoje)

In [28]:
estado_amanha = DiscreteDistribution( { 'prob_subir': 0.5, 'prob_descer': 0.5 } )

In [31]:
#print(estado_amanha)

In [1]:
tabela = ConditionalProbabilityTable(
    [
        [ 'prob_subir', 'prob_subir', 0.0 ],
        [ 'prob_subir', 'prob_descer', 0.5 ],
        [ 'prob_descer', 'prob_descer', 0.5 ],
        [ 'prob_descer', 'prob_subir', 0.0 ],
    ],
    [estado_hoje, estado_amanha]
)

NameError: name 'ConditionalProbabilityTable' is not defined